In [14]:
import gym
import collections
import numpy as np
import tensorflow as tf
from random import sample
from tensorflow.keras import layers, Model

print(tf.config.list_physical_devices('GPU')[0])

experience_buffer = collections.deque([])
REAPLAY_MEMORY_SIZE = 10000 # should be 1000000

env = gym.make("ALE/Breakout-v5")

N_ACTIONS = env.action_space.n
OBS_RANGE = int(env.observation_space.high_repr) - int(env.observation_space.low_repr)
OBS_SHAPE = env.observation_space.shape
# SKIP_FRAMES = [2, 3, 4] # possible addition --> skip randomly 2/3/4 frames instead of always 4
N_INPUT_FRAMES = 4

REPLAY_START_SIZE = 500 # should be 50000

BATCH_SIZE = 32

input_shape = [*OBS_SHAPE]
input_shape[-1] *= N_INPUT_FRAMES

obs = np.zeros(input_shape)

model = tf.keras.applications.efficientnet.EfficientNetB0(
    include_top=True,
    weights=None,
    input_shape=input_shape,
    classes=N_ACTIONS,
    classifier_activation='softmax'
)

total_params = sum([np.prod(w.get_shape().as_list()) for w in model.trainable_weights])
print(total_params)


PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


/home/marcello/github/AtariGym/env/lib/python3.8/site-packages/keras/applications/efficientnet.py:288: UserWarning: This model usually expects 1 or 3 input channels. However, it was passed an input_shape with 12 input channels.
  input_shape = imagenet_utils.obtain_input_shape(


4015264


In [15]:
def preprocess_observation(obs):
    return obs / OBS_RANGE

In [ ]:
def simulate(action):
    i = 0
    k_reward = 0
    done = 0
    while i < N_INPUT_FRAMES and not done:

        observation, reward, done, info = env.step(action)
        obs[:, :, i:i+3] = observation
        i += 1
        k_reward += reward

    p_obs = preprocess_observation(obs)
    return p_obs, k_reward, done

In [22]:
counter_replay_start = 0

for _ in range(int(1e6)):                           # n° of episodes

      observation, info = env.reset(seed=42, return_info=True)
      action = env.action_space.sample() # how do you solve the first frame? which action do you choose? (since the obs are 4 frames concat)

      for _ in range(int(1e3)):                     # n° of frames per episode

            i = 0
            k_reward = 0
            done = 0
            # sample_k_frames = sample(SKIP_FRAMES, 1)[0]
            
            while i < N_INPUT_FRAMES and not done:

                  observation, reward, done, info = env.step(action)
                  obs[:, :, i:i+3] = observation
                  i += 1
                  k_reward += reward

            p_obs = preprocess_observation(obs)

            if counter_replay_start > REPLAY_START_SIZE:
                  sample(experience_buffer, BATCH_SIZE)

            else:
                  counter_replay_start += 1

            if counter_replay_start < REPLAY_START_SIZE:
                  # random sampling the first "REPLAY_START_SIZE" steps
                  action = env.action_space.sample()
                  counter_replay_start += 1
                  next_p_obs, reward, is_end_state = simulate(action) 
                  experience_buffer.append((p_obs, action, reward, next_p_obs, is_end_state))
            else:
                  # policy
                  if np.random.uniform() < eps:
                        action = env.action_space.sample()
                  else:
                        action = max(model(p_obs))
            print(action)

            observation, reward, done, info = env.step(0)

            if done:
                observation, info = env.reset(return_info=True)
            # break
            break
      break

env.close()

ValueError: Input 0 of layer "efficientnetb0" is incompatible with the layer: expected shape=(None, 210, 160, 12), found shape=(210, 160, 12)